In [2]:
from dotenv import load_dotenv
load_dotenv()
import sys 
sys.path.append("..")

In [4]:
from rag.document_loader import best_loader
loaders = best_loader("../data/2024 주택청약 FAQ.pdf")

DoclingLoader 사용


2025-10-20 23:44:19,342 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-20 23:44:19,401 - INFO - Going to convert document batch...
2025-10-20 23:44:19,401 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 4f2edc0f7d9bb60b38ebfecf9a2609f5
2025-10-20 23:44:19,424 - INFO - Loading plugin 'docling_defaults'
2025-10-20 23:44:19,425 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-10-20 23:44:19,425 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-20 23:44:19,429 - INFO - Loading plugin 'docling_defaults'
2025-10-20 23:44:19,431 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-10-20 23:44:19,431 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-10-20 23:44:19,969 - INFO - Auto OCR model selected ocrmac.
2025-10-20 23

DoclingLoader 실패: Invalid code point
PDFPlumberLoader 사용
PDFPlumberLoader 실패: Cannot convert None to Decimal.
PyMuPDFLoader 사용


In [ ]:
print(loaders[107].page_content)

In [ ]:
from rag.chunker import adaptive_chunker
chunks = adaptive_chunker(loaders)